# Dimcustomer

In [0]:

create or replace table datamodelling_lak.gold.Dimcustomers as
(
with re_dup as
(
  select
  distinct(customer_id),
  customer_name,
  customer_email,
  upper_customer_name
from 
  datamodelling_lak.silver.silver_table)
select *,row_number() over(order by customer_id) as DimCustomerKey from re_dup
)
--Here are the DataFrames in the session:

# Dimproducts Table

In [0]:
create or replace table datamodelling_lak.gold.dimproducts as
(
select *,
row_number() over (order by product_id) as DimProductKey 
from
(
  select 
    distinct(product_id),
    product_name,
    product_category
  from 
    datamodelling_lak.silver.silver_table
)
)

In [0]:
select * from datamodelling_lak.gold.dimproducts

In [0]:
select * from datamodelling_lak.silver.silver_table

# Dimpayments

In [0]:

create or replace table datamodelling_lak.gold.dimpayments as
(
with cte_payments as 
(
  select distinct(payment_type) from datamodelling_lak.silver.silver_table
)
select *,row_number() over(order by payment_type) as DimPaymentKey from cte_payments
)

# Dimcountry

In [0]:
create or replace table datamodelling_lak.gold.dimcountry as
(
select *, row_number() over(order by country) as DimCountryKey
from
(
  select distinct(country) from datamodelling_lak.silver.silver_table
)
)

# parent dimension dimsales

In [0]:
%python
spark.sql(f""" select * from datamodelling_lak.silver.silver_table""").columns
  

In [0]:
create or replace table datamodelling_lak.gold.dimsales as
(
with sales_cte as
(
select 
  order_id,
 order_date,
 customer_id,
 customer_name,
 customer_email,
 product_id,
 product_name,
 product_category,
 payment_type,
 country,
 last_updated,
 upper_customer_name,
 process_date
 from datamodelling_lak.silver.silver_table
)

select *, row_number() over(order by order_id) as DimSalesKey from sales_cte
)

In [0]:
select * from datamodelling_lak.gold.dimsales

# Fact Table

In [0]:
create or replace table datamodelling_lak.gold.FactSales
(
select 
  F.unit_price,
  F.quantity,
  C.DimCustomerKey,
  P.DimProductKey,
  D.DimPaymentKey,
  S.DimSalesKey,
  E.DimCountryKey
from datamodelling_lak.silver.silver_table F 
left join datamodelling_lak.gold.dimcustomers C
on F.customer_id = C.customer_id
left join datamodelling_lak.gold.dimproducts P
on F.product_id = P.product_id
left join datamodelling_lak.gold.dimpayments D
on F.payment_type =D.payment_type
left join datamodelling_lak.gold.dimcountry E
on F.country = E.country
left join datamodelling_lak.gold.dimsales S
on F.order_id = S.order_id
)